In [1]:
# Import dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

In [12]:
import requests

Key = '06e95084cb5d3426c57b51987827eb7d'
url = "https://developers.zomato.com/api/v2.1/categories"

if __name__ == '__main__':
    r = requests.get(url, headers={'user-key': Key})
    if r.ok:
        data = r.json()
        print(data)

{'categories': [{'categories': {'id': 1, 'name': 'Delivery'}}, {'categories': {'id': 2, 'name': 'Dine-out'}}, {'categories': {'id': 3, 'name': 'Nightlife'}}, {'categories': {'id': 4, 'name': 'Catching-up'}}, {'categories': {'id': 5, 'name': 'Takeaway'}}, {'categories': {'id': 6, 'name': 'Cafes'}}, {'categories': {'id': 7, 'name': 'Daily Menus'}}, {'categories': {'id': 8, 'name': 'Breakfast'}}, {'categories': {'id': 9, 'name': 'Lunch'}}, {'categories': {'id': 10, 'name': 'Dinner'}}, {'categories': {'id': 11, 'name': 'Pubs & Bars'}}, {'categories': {'id': 13, 'name': 'Pocket Friendly Delivery'}}, {'categories': {'id': 14, 'name': 'Clubs & Lounges'}}]}


In [15]:
params= {
    'user-key': Key,
    "category": 3,
    "city": 280,
    "entity_type": "city",
}

url = "https://developers.zomato.com/api/v2.1/categories"

headers= {
    "user-key": "06e95084cb5d3426c57b51987827eb7d"
}
results = requests.get(url,headers)

In [16]:
results.json()

{'code': 403, 'status': 'Forbidden', 'message': 'Invalid API Key'}

In [ ]:
# Connect to BeautifulSoup and define the parser as HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find all apts that fit the specified criteria and # of apts
apts = soup.find_all('p', attrs={'class':'result-info'})
print(len(apts))

##  Parse through Craigslist results

In [ ]:
# results are returned as an iterable list
results = soup.find_all('li', class_="result-row")
list_results = []


# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_="result-title").text
        # Identify and return price of listing
        price = result.a.span.text.strip('$')
        # Identify number of rooms
        n_rooms = result.find('span', class_ = 'housing').text.split()
        # Identify and return link to listing
        link = result.a['href']
        # Identify posting date
        post_time = result.find('time')['datetime']
        post_time = pd.to_datetime(post_time)
        # Identify neighborhood
        neighborhood = result.find('span', class_= 'result-hood').text
        
        list_results.append({'#bedrooms': n_rooms, '$price': price,
                        'Post_Title': title, 'URL': link, 'Post_Time': post_time, 'Neighborhood': neighborhood})
        
        # Print results only if title, price, and link are available
        if (title and price and link):
            print('-------------')
            print(title)
            print(price)
            print(n_rooms)
            print(link)
            print(post_time)
            print(neighborhood)
    except AttributeError as e:
        print(e)
        

## Create DataFrame from results

In [ ]:
# Create DataFrame
apt_df = pd.DataFrame(list_results)

# Clean data and remove text for room/ 
bedrooms = apt_df['#bedrooms']
rooms = []

for room in bedrooms:
    rooms.append(room[0].strip('br'))

apt_df['#rooms'] = rooms
apt_df['Post_Time'] = apt_df['Post_Time'].dt.date

apt_df

In [ ]:
# Clean data and remove text from #bedrooms and create new column for sqft 

# Split before and after the dash & remove the 'ft2' text
apt_df['#bedrooms'] = apt_df['#bedrooms'].astype(str)
apt_df['#bedrooms'] = apt_df['#bedrooms'].str.split(',').str[2]
apt_df['#bedrooms'].fillna("", inplace=True)
apt_df['#bedrooms'] = apt_df['#bedrooms'].str.split('ft2') 

###############################################################

# Loop through the new bedroom values and remove remaining text that will prevent converting to int
bedrooms = apt_df['#bedrooms']
rooms = []

for room in bedrooms:
    rooms.append(room[0].strip(','))

apt_df['sqft'] = rooms
apt_df['sqft'] = apt_df['sqft'].str.replace(r"'", '')
apt_df['sqft'] = apt_df['sqft'].astype(str)
apt_df['sqft'] = apt_df['sqft'].str.strip()

##############################################################

# Convert values to int and if NAN make = 0
sqfts = apt_df['sqft']
number = []

for sqft in sqfts:
    number.append(sqft)

newsqft = pd.to_numeric(number, errors='ignore')
apt_df['sqft'] = newsqft
apt_df['sqft'].fillna("0", inplace=True)
apt_df['sqft'] = apt_df['sqft'].astype(int)

##############################################################

# Convert price and # rooms to int in order to graph data
apt_df['$price'] = apt_df['$price'].astype(int)
apt_df['#rooms'] = apt_df['#rooms'].astype(int)


##############################################################

# Clean up the Neighborhood name
apt_df['Hood'] = apt_df['Neighborhood'].str.extract(r"\((.*?)\)", expand=False)
apt_df['Hood'] = apt_df['Hood'].astype(str)
apt_df['Hood'] = apt_df['Hood'].str.strip()

neighborhoods = apt_df["Hood"].unique()
new_hoods = ['North Beach', 'Lower Nob Hill', 'Nob Hill', 'Russian Hill', 'Lower Pac Hts', 'Pac Heights']
my_dict = dict(zip(neighborhoods, new_hoods))

apt_df['Hood'].replace(my_dict, inplace=True)


apt_df

In [ ]:
# New DataFrame to build graphs in
apts_data = apt_df[['#rooms', 'sqft', '$price', 'Hood', 'Post_Time', 'Post_Title', 'URL']]

apts_data.dtypes

# Send data to SlackBot

In [ ]:
import os
from slackclient import SlackClient
import pprint
from collections import Counter

In [ ]:
slack_data = apts_data.to_dict('records')
str(slack_data)

In [ ]:
# Prints the nicely formatted dictionary
pprint.pprint(slack_data)

# Sets 'pretty_dict_str' to 
pretty_dict_str = pprint.pformat(slack_data)

In [ ]:
SLACK_TOKEN = "ENTER_SLACK_API"
SLACK_CHANNEL = "ENTER_SLACK_CHANNEL"

sc = SlackClient(SLACK_TOKEN)

sc.api_call(
    "chat.postMessage", channel=SLACK_CHANNEL, text=pretty_dict_str,
    username='pybot', icon_emoji=':robot_face:'
)